In [25]:
from dtOmitted import build_tree,DecisionNode
import numpy as np
from collections import defaultdict
from decision_tree import DecisionTree
import matplotlib.pyplot as plt
from random_forest import RandomForest
from logistic import logistic
import runner


In [4]:
filename = 'SPECTF.dat'
data = np.loadtxt(filename, delimiter=',')
X = data[:, 1:]
y = np.array([data[:, 0]]).T
n, d = X.shape
num_folds=10
    


In [5]:
idx = np.arange(n)
np.random.seed(10)
np.random.shuffle(idx)
X = X[idx]
y = y[idx]

Xtrain = X[:n-n//num_folds, :]  # train on first 100 instances
Xtest = X[n-n//num_folds:, :]
ytrain = y[:n-n//num_folds, :]  # test on remaining instances
ytest = y[n-n//num_folds:, :]

rf_classifier=RandomForest(15,100,0.3)
rf_classifier.fit(Xtrain,ytrain)

l_classifier=logistic()
l_classifier.fit(Xtest,ytest)
        

In [6]:
idx=-4
print("X={0} \nY={1}".format(X[idx],y[idx]))

X=[ 77.  59.  76.  78.  40.  34.  75.  63.  65.  26.  70.  28.  50.  46.  25.
  66.  46.  52.  63.  63.  68.  56.  46.  47.  12.   6.  28.  23.  56.  48.
  68.  55.  63.  54.  40.  20.  65.  68.  56.  57.   5.  10.  13.  10.] 
Y=[ 1.]


In [7]:
rf_classifier.predict([X[idx]])

([1.0], [1.0])

In [8]:
l_classifier.predict([X[idx]])

[1]

In [20]:
idx=-9
print("X={0} \nY={1}".format(X[idx],y[idx]))

X=[ 58.  63.  80.  71.  76.  70.  70.  71.  64.  63.  74.  78.  77.  75.  62.
  61.  62.  56.  71.  52.  82.  71.  84.  85.  71.  71.  57.  47.  42.  39.
  70.  70.  50.  70.  50.  46.  58.  60.  76.  73.  82.  77.  65.  66.] 
Y=[ 0.]


In [21]:
rf_classifier.predict([X[idx]])

([0.0], [0.66666666666666663])

In [22]:
l_classifier.predict([X[idx]])

[1]